### Condition (첫사용시 selenium이랑 크롬 드라이브 설치해야됨!!)

### 1. 순위
### 2. 썸네일
### 3. 곡정보
### 4. 가수명

### 을 멜론에서 추출해서 excel로 저장하기

### My Code

In [109]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os

driver = webdriver.Chrome('c:/chromedriver.exe')
url = 'https://www.melon.com/chart/day/index.htm'
# url= 'https://www.melon.com/chart/month/index.htm'
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html,'html.parser')

In [110]:
melon_ranks = soup.select('td div.wrap span.rank')            # 순위 추출
melon_titles = soup.select('div.rank01 span a')               # 타이틀 추출
melon_artists = soup.select('div.rank02 span a')              # 아티스트 추출

In [111]:
melon_images_elements = soup.select('td div.wrap a.image_typeAll img') # 썸네일 추출

folder = "C:/myPyCode/Malon2"

if not os.path.exists(folder) :
    os.makedirs(folder)

# c = []
# res  = []
for melon_images_html in melon_images_elements  :
    A = melon_images_html.get('src')
    B = A.split('/melon/resize/120/quality/80/optimize')
    front_image_file_name = B[0]
#     c.append(front_image_file_name)
    # print(front_image_file_name)
    image_file_name = os.path.basename(front_image_file_name)
    image_path = os.path.join(folder, image_file_name)
    imageFile = open(image_path, 'wb')

    html_image = requests.get(front_image_file_name)
#     res.append(html_image)

    chunk_size = 1000000
    for chunk in html_image.iter_content(chunk_size) :
        imageFile.write(chunk)
    imageFile.close()

#     os.listdir(folder)

In [ ]:
print(len(c), len(res))
import collections
collections.Counter([i.get('src')for i in melon_images_elements])

### Another Code

In [ ]:
from bs4 import BeautifulSoup
import openpyxl
import requests
import os
from io import BytesIO
import requests
import xlsxwriter
from selenium import webdriver
driver = webdriver.Chrome("C:/playwithdata/chromedriver.exe")
# driver = webdriver.Chrome("C:/Users/Playdata/Desktop/chromedriver.exe")
url = 'https://www.melon.com/new/index.htm#params%5BareaFlg%5D=I&po=pageObj&startIndex=1'
driver.implicitly_wait(1)
driver.get(url)
html = driver.page_source
workbook = xlsxwriter.Workbook('C:/Users/Playdata/Desktop/melon.xlsx')
ws = workbook.add_worksheet()

ws.write(0, 0, 'No')
ws.write(0, 2, '곡정보')
ws.write(0, 3, '가수')
ws.write(0, 4, '앨범')
soup = BeautifulSoup(html,"html.parser")
sub = soup.select(".ellipsis.rank01") # 노래명
sub2 = soup.select(".ellipsis.rank02") # 가수명
sub3 = soup.select(".ellipsis.rank03") # 앨범명
images = soup.select("tbody img")
ws.set_default_row(120) # row height 설정

for i in range(len(sub2)):
    p1=sub[i].get_text().strip().split("\n")[0]
    p2=sub2[i].get_text().strip().split("\n")[0]
    p3=sub3[i].get_text().strip().split("\n")[0]
    ws.write(i+1, 0, i+1) 
    ws.write(i+1, 2, p1) 
    ws.write(i+1, 3, p2[:len(p2)//2])
    ws.write(i+1, 4, p3) 
    ws.set_column(i+1, 1, 30) 
i=1
for p in images:
    p=p.get("src")
    html_image = requests.get(p)
    res = requests.get(p)
    image_data = BytesIO(res.content)
    image_size = len(image_data.getvalue())
    if image_size > 0: 
        ws.insert_image('B'+str(i+1), url, {'image_data': image_data}) 
    i+=1
workbook.close()